<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Algorithmic Trading 

**Chapter 07 &mdash; Working with Real-Time Data and Sockets**

## Visualizing Streaming Data with Plotly

### The Basics

### Requirements

```sh
pip install plotly ipywidgets

# install jupyter lab extensions
jupyter labextension install jupyterlab-plotly
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install plotlywidget
```

In [ ]:

import zmq
from datetime import datetime
import plotly.graph_objects as go

In [ ]:
symbol = 'SYMBOL'

In [ ]:
fig = go.FigureWidget()
fig.add_scatter()
fig

In [ ]:
context = zmq.Context()
socket = context.socket(zmq.SUB)
socket.connect('tcp://0.0.0.0:5555')
socket.setsockopt_string(zmq.SUBSCRIBE, 'SYMBOL')

In [ ]:
times = list()
prices = list()

In [ ]:
for _ in range(50):
    msg = socket.recv_string()
    t = datetime.now()
    times.append(t)
    _, price = msg.split()
    prices.append(float(price))
    fig.data[0].x = times
    fig.data[0].y = prices

In [ ]:
fig = go.FigureWidget()
fig.add_scatter(name='SYMBOL')
fig.add_scatter(name='SMA1', line=dict(width=1, dash='dot'),
                mode='lines+markers')
fig.add_scatter(name='SMA2', line=dict(width=1, dash='dash'),
                mode='lines+markers')
fig

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()

In [ ]:
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    df = df.append(pd.DataFrame({sym: float(price)}, index=[t]))
    df['SMA1'] = df[sym].rolling(5).mean()
    df['SMA2'] = df[sym].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['SMA1']
    fig.data[2].y = df['SMA2']

In [ ]:
from plotly.subplots import make_subplots

In [ ]:
f = make_subplots(rows=3, cols=1, shared_xaxes=True)
f.append_trace(go.Scatter(name='SYMBOL'), row=1, col=1)
f.append_trace(go.Scatter(name='RETURN', line=dict(width=1, dash='dot'),
                mode='lines+markers', marker={'symbol': 'triangle-up'}),
                row=2, col=1)
f.append_trace(go.Scatter(name='MOMENTUM', line=dict(width=1, dash='dash'),
                mode='lines+markers', marker={'symbol': 'x'}), row=3, col=1)
# f.update_layout(height=600)

In [ ]:
fig = go.FigureWidget(f)

In [ ]:
fig

In [ ]:
import numpy as np

In [ ]:
df = pd.DataFrame()

In [ ]:
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    df = df.append(pd.DataFrame({sym: float(price)}, index=[t]))
    df['RET'] = np.log(df[sym] / df[sym].shift(1))
    df['MOM'] = df['RET'].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['RET']
    fig.data[2].y = df['MOM']

In [ ]:
socket = context.socket(zmq.SUB)

In [ ]:
socket.connect('tcp://0.0.0.0:5556')

In [ ]:
socket.setsockopt_string(zmq.SUBSCRIBE, '')

In [ ]:
for _ in range(5):
    msg = socket.recv_string()
    print(msg)

In [ ]:
fig = go.FigureWidget()
fig.add_bar()
fig

In [ ]:
x = list('abcdefgh')
fig.data[0].x = x
for _ in range(100):
    msg = socket.recv_string()
    y = msg.split()
    y = [float(n) for n in y]
    fig.data[0].y = y

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>